In [ ]:
import pandas as pd
import numpy as np

# API access
import requests
import warnings

# Progress bar and timing
from tqdm import tqdm
tqdm.pandas(desc="Progress")

warnings.simplefilter(action='ignore', category=FutureWarning)

### NMB revisions March 14, 2022: 

  * Commenting out some sections so we can collect handbooks from all schools rather than just middle and high schools. 
  * Adding the NCES school ID (NCESSCH) so we have a unique identifier for each school
  * Adding school type (SCH_TYPE_TEXT) so we can filter on regular schools
  * Removing grade-specific filters
  * Importing revised dataset to account for 11 records that had to be manually fixed
  * Setting import datatype to str so IDs don't get mangled into integers
  * Keeping original column names throughout (eventually they will be matched back with original dataset)
  * Writing a function to replace logic that selects website from multiple columns

### Function definitions

In [ ]:
def session_request(url_list):
    '''
    Retrieves json-formatted web server responses for a list of urls
    Accepts: list of str
    Returns: list of dict (json objects)
    '''
    results = []
    session = requests.Session()
    for url in tqdm(url_list):
        response = session.request('GET', url)
        results.append(response.json())
    return results

### School-level source data from NCES

In [ ]:
# Importing the list of schools (note: 11 records manually updated due to shifted columns)
school = pd.read_csv('./files/ccd_1819_directory_rev.csv', low_memory=False, dtype=str,
                    usecols=['STATENAME', 'NCESSCH', 'SCH_NAME', 'LSTREET1', 'LZIP', 'WEBSITE',
                            'SCH_TYPE_TEXT'])

In [ ]:
school.shape

### Gathering Place IDs from Google Place API

In [ ]:
# API key obtained via Google Cloud Console under project gcp-gu-ppalab
local_file = '/Users/nb775/auth/brodnax_places_auth.txt'
with open(local_file) as txtfile:
    my_key = txtfile.read().strip('\n')

In [ ]:
# Filter only regular schools
school[['NCESSCH', 'SCH_TYPE_TEXT']].groupby(['SCH_TYPE_TEXT']).count()

In [ ]:
place_id = school[school['SCH_TYPE_TEXT']=='Regular School'].copy()

In [ ]:
place_id.shape

In [ ]:
place_id.head()

In [ ]:
# Creating a search term for the Google Maps API
g_place = list(place_id['SCH_NAME']+'%20'+place_id['LSTREET1']+'%20'+place_id['LZIP'])

In [ ]:
len(g_place)

In [ ]:
# Replacing the spaces in the search term with '%20' in order to make it compatible with the API
place = []
for term in g_place:
    if isinstance(term, str):
        no_space = term.replace(' ', '%20')
        place.append(no_space)
    else:
        place.append('')

In [ ]:
# Setting the urls for the API 
pid_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input="
pid_param = '&inputtype=textquery&fields=place_id&key=' + my_key

In [ ]:
# Generating a unique url for each school in order to feed that into the API
pid_api = []
for loc in place:
    pid_api.append(pid_url + loc + pid_param)

In [ ]:
pid_list = session_request(pid_api)

In [ ]:
# Visualizing the errors in order to better understand the results
pid_results = [len(result.get('candidates')) for result in pid_list]
pid_status = [result['status'] for result in pid_list]
pid_error = [result.get('error_message') for result in pid_list]
pid_responses = pd.DataFrame({'num_results':pid_results, 'status':pid_status, 'error':pid_error})

# Export the errors for analysis
pid_responses[pid_responses['num_results'] != 1].to_csv('./files/pid_responses.csv')

In [ ]:
# Extract place IDs from results, keeping the same structure as the original dataframe
only_pid = []

for result in pid_list:
    result_list = result.get('candidates')
    if len(result_list) == 1:
        only_pid.append(result_list[0].get('place_id'))
    elif len(result_list) == 2: # If there are 2 place ids for one school I am wrapping the two place_ids in the following format (place_id 1, place_id 2)
        only_pid.append((result_list[0].get('place_id'), result_list[1].get('place_id')))
    else:
        only_pid.append(np.nan)

In [ ]:
len(only_pid)

In [ ]:
# Adding the place_id to the original dataframe
place_id['g_pid'] = only_pid

In [ ]:
# Duplicating the rows with 2 place_ids
place_id = place_id.explode('g_pid')

# Replacing the nan values in 'g_pid' as 'None'
place_id['g_pid'] = place_id['g_pid'].fillna('None')

# Resaving the new list of place_ids to only_pid
only_pid = list(place_id['g_pid'])

# Checking the shape to make sure that only the rows with 2 place_ids got duplicated
place_id.shape

### Exporting data for use in Part 3

In [ ]:
# Export data - to be used as input for "website_crawler_google_place_websites.ipynb"
place_id.to_csv('./files/place_ids.csv', index=False)